In [1]:
!pip install ipywidgets --quiet
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
import ipywidgets as widgets
import matplotlib.colors as mcolors


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.0 MB/s eta 0:00:00


In [2]:
def get_neighbors_1d(index, grid, radius):
    n = len(grid)
    return [grid[(index + i) % n] for i in range(-radius, radius+1)]

def get_neighbors_2d(i, j, grid, neighborhood="Moore"):
    rows, cols = grid.shape
    neighbors = []
    offsets = []

    if neighborhood == "Moore":
        offsets = [(-1, -1), (-1, 0), (-1, 1),
                   (0, -1),           (0, 1),
                   (1, -1),  (1, 0),  (1, 1)]
    elif neighborhood == "VonNeumann":
        offsets = [(-1, 0), (0, -1), (0, 1), (1, 0)]

    for dx, dy in offsets:
        ni, nj = (i + dx) % rows, (j + dy) % cols
        neighbors.append(grid[ni, nj])

    return neighbors


In [3]:
def rule_30(neighborhood):
    # neighborhood: list of [left, center, right]
    binary = ''.join(str(int(x)) for x in neighborhood)
    rule_bin = {
        "111": 0, "110": 0, "101": 0, "100": 1,
        "011": 1, "010": 1, "001": 1, "000": 0
    }
    return rule_bin.get(binary, 0)

def game_of_life(cell, neighbors):
    alive = sum(neighbors)
    if cell == 1:
        return 1 if 2 <= alive <= 3 else 0
    else:
        return 1 if alive == 3 else 0


In [4]:
def simulate_1d(grid, steps, rule_fn, radius):
    history = [grid.copy()]
    for _ in range(steps):
        new_grid = np.zeros_like(grid)
        for i in range(len(grid)):
            n = get_neighbors_1d(i, grid, radius)
            new_grid[i] = rule_fn(n)
        history.append(new_grid)
        grid = new_grid
    return history

def simulate_2d(grid, steps, rule_fn, neighborhood="Moore"):
    history = [grid.copy()]
    for _ in range(steps):
        new_grid = np.zeros_like(grid)
        for i in range(grid.shape[0]):
            for j in range(grid.shape[1]):
                neighbors = get_neighbors_2d(i, j, grid, neighborhood)
                new_grid[i, j] = rule_fn(grid[i, j], neighbors)
        history.append(new_grid)
        grid = new_grid
    return history


In [5]:
def plot_1d(history):
    data = np.array(history)
    plt.figure(figsize=(12, 6))
    plt.imshow(data, cmap='binary', interpolation='nearest')
    plt.title("1D Cellular Automaton Evolution")
    plt.xlabel("Κυψέλες")
    plt.ylabel("Χρόνος")
    plt.show()

def plot_2d(history):
    for i, frame in enumerate(history):
        plt.figure(figsize=(5, 5))
        plt.imshow(frame, cmap='viridis')
        plt.title(f"2D - Βήμα {i}")
        plt.axis('off')
        plt.show()


In [6]:
def run_gui():
    mode = widgets.ToggleButtons(options=["1D", "2D"], description="Διάσταση:")
    rule = widgets.Dropdown(options=["Rule 30", "Game of Life"], description="Κανόνας:")
    size = widgets.IntSlider(value=50, min=10, max=200, description="Μέγεθος:")
    steps = widgets.IntSlider(value=20, min=1, max=100, description="Βήματα:")
    button = widgets.Button(description="Εκκίνηση")
    output = widgets.Output()

    def on_click(b):
        with output:
            output.clear_output()
            if mode.value == "1D":
                grid = np.zeros(size.value)
                grid[size.value // 2] = 1
                rule_fn = rule_30 if rule.value == "Rule 30" else lambda n: 0
                history = simulate_1d(grid, steps.value, rule_fn, radius=1)
                plot_1d(history)
            else:
                grid = np.random.choice([0, 1], size=(size.value, size.value))
                rule_fn = game_of_life if rule.value == "Game of Life" else lambda c, n: 0
                history = simulate_2d(grid, steps.value, rule_fn)
                plot_2d(history)

    button.on_click(on_click)
    display(widgets.VBox([mode, rule, size, steps, button, output]))

run_gui()
